In [1]:
import requests
import json
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from datetime import datetime
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")
if not API_KEY:
    raise Exception("GOOGLE_API_KEY not found in .env file")

In [ ]:
# geocoding the address
def geocode_address(address, api_key):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"
    response = requests.get(url)
    data = response.json()
    
    if data["status"] == "OK" and len(data["results"]) > 0:
        location = data["results"][0]["geometry"]["location"]
        return location["lat"], location["lng"]
    else:
        raise Exception(f"Geocoding failed for {address}: {data.get('status', 'Unknown error')}")

In [ ]:
# taking weather into consideration
def get_weather_multiplier(lat, lng, api_key):
    url = f"https://weather.googleapis.com/v1/current?lat={lat}&lng={lng}&key={api_key}"
    response = requests.get(url)
    data = response.json()
    
    if "currentConditions" in data and len(data["currentConditions"]) > 0:
        conditions = data["currentConditions"][0]
        precip_prob = float(conditions.get("precipitationProbability", 0)) / 100.0
        precip_type = conditions.get("precipitationType", [])
        multiplier = 1.0 + 0.2 * precip_prob  
        if "SNOW" in precip_type or "FREEZING_RAIN" in precip_type:
            multiplier += 0.1  
        return multiplier
    return 1.0  

In [5]:
# taking traffic into consideration based on weather
def get_distance_matrix(locations, api_key, weather_multipliers):
    url = "https://maps.googleapis.com/maps/api/distancematrix/json"
    origins = "|".join([f"{lat},{lng}" for lat, lng in locations])
    destinations = origins
    params = {
        "origins": origins,
        "destinations": destinations,
        "key": api_key,
        "mode": "driving",
        "departure_time": int(datetime.now().timestamp())  # Current time for traffic
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    if data["status"] != "OK":
        raise Exception(f"API Error: {data['status']}")
    
    distance_matrix = []
    for i, row in enumerate(data["rows"]):
        row_durations = []
        for element in row["elements"]:
            if element["status"] == "OK":
                duration = (element.get("duration_in_traffic") or element["duration"])["value"]
                adjusted_duration = int(duration * weather_multipliers[i])
                row_durations.append(adjusted_duration)
            else:
                raise Exception(f"Distance Matrix Error: {element['status']}")
        distance_matrix.append(row_durations)
    return distance_matrix

In [6]:
# vechile routing problem data model
def create_data_model(distance_matrix, locations):
    data = {}
    data["distance_matrix"] = distance_matrix
    data["num_vehicles"] = 1
    data["warehouse"] = 0  #starting point
    data["locations"] = locations
    return data

In [7]:
# final optimal route
def get_solution(data, manager, routing, solution):
    route = []
    total_time = 0
    index = routing.Start(0)
    while not routing.IsEnd(index):
        node = manager.IndexToNode(index)
        route.append(data["locations"][node])
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        total_time += routing.GetArcCostForVehicle(previous_index, index, 0)
    
    print(f"Optimal route time: {total_time / 60:.2f} minutes")
    print("Route order (node indices):")
    for i, loc in enumerate(route):
        print(f"Stop {i}: {loc}")
    return route

In [8]:
# solving vehicle routing problem
def solve_vrp(addresses, api_key):

    # geocode addresses to lat/lng
    locations = []
    for address in addresses:
        lat, lng = geocode_address(address, api_key)
        locations.append([lat, lng])
    
    # fetch weather multipliers
    weather_multipliers = []
    for lat, lng in locations:
        multiplier = get_weather_multiplier(lat, lng, api_key)
        weather_multipliers.append(multiplier)
    
    # get adjusted distance matrix
    distance_matrix = get_distance_matrix(locations, api_key, weather_multipliers)
    
    # create data model
    data = create_data_model(distance_matrix, locations)
    
    # create routing index manager
    manager = pywrapcp.RoutingIndexManager(len(data["distance_matrix"]), 
                                         data["num_vehicles"], 
                                         data["depot"])
    
    # create routing model
    routing = pywrapcp.RoutingModel(manager)
    
    # define cost of each arc
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    # set search parameters
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    # solve the problem
    solution = routing.SolveWithParameters(search_parameters)
    
    if solution:
        return get_solution(data, manager, routing, solution)
    else:
        raise Exception("No solution found")

In [12]:
# Cell 8: Example usage
if __name__ == "__main__":
    # Example addresses of Croma Stores in Vadodara (corrected)
    addresses = [
        "Ankhol, Waghodia Road, Vadodara, Gujarat 390019",  # Warehouse
        "Glade Centrum, New VIP Road, Vadodara, Gujarat 390022",  # Croma New VIP Road
        "Sakar Whizzo, Lalbaug Road, Manjalpur, Vadodara, Gujarat 390011",  # Croma Manjalpur
        "Centre Square Mall, Sarabhai Road, Subhanpura, Vadodara, Gujarat 390007",  # Croma Subhanpura
        "Bricklane, VIP Road, Karelibagh, Vadodara, Gujarat 390018",  # Croma Karelibagh
        "The Iconic, Harni Road, Vadodara, Gujarat 390022"  # Croma Harni
    ]
    
    optimal_route = solve_vrp(addresses, API_KEY)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)